In [4]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/8 - limpeza das variáveis/cs.csv')

# Analisar TODAS as variáveis numéricas (incluindo as com missing)
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
# Remover target e ID
numeric_cols = [col for col in numeric_cols if col not in ['aleitamento_materno_exclusivo', 'id_anon']]

print("ANÁLISE DE RANGES E OUTLIERS - TODAS AS VARIÁVEIS NUMÉRICAS")
print("="*80)

for col in numeric_cols:
    print(f"\n{col}:")
    print(f"  Count: {df[col].count()} (de {len(df)} - {df[col].isnull().sum()} missing)")
    
    if df[col].count() > 0:  # Se tem dados não-missing
        print(f"  Min: {df[col].min()}")
        print(f"  Max: {df[col].max()}")
        print(f"  Mean: {df[col].mean():.2f}")
        print(f"  Median: {df[col].median():.2f}")
        print(f"  Std: {df[col].std():.2f}")
        
        # Detectar possíveis códigos suspeitos
        unique_values = df[col].dropna().unique()
        
        # Verificar valores que podem ser códigos (99, 999, -9, etc.)
        suspicious_values = []
        for val in unique_values:
            if val in [99, 999, 9999, -9, -99, -999]:
                suspicious_values.append(val)
            elif val > 1000 and col not in ['h02_peso', 'h03_altura']:  # Exceto peso/altura que podem ser grandes
                suspicious_values.append(val)
        
        if suspicious_values:
            print(f"  ⚠️  VALORES SUSPEITOS: {suspicious_values}")
            for sus_val in suspicious_values:
                count_sus = (df[col] == sus_val).sum()
                print(f"      {sus_val}: {count_sus} ocorrências ({count_sus/df[col].count()*100:.1f}%)")
        
        # Mostrar distribuição dos valores únicos se poucos valores
        if len(unique_values) <= 10:
            print(f"  Valores únicos: {sorted(unique_values)}")
            value_counts = df[col].value_counts().head(5)
            print(f"  Top 5 valores: {value_counts.to_dict()}")
    
    print("-" * 50)

# Análise específica das variáveis com missing
print("\n\nANÁLISE ESPECÍFICA - VARIÁVEIS COM MISSING VALUES")
print("="*80)

missing_vars = ['vd_imc_mae', 'vd_anthro_zwfl', 'vd_zimc', 'k08_quilos', 
                'k06_peso_engravidar', 'k07_peso_final', 'k05_prenatal_consultas', 
                'k04_prenatal_semanas', 'tempo_primeira_mamada_horas', 
                'k01_gestacoes', 'k02_filhos_vivos', 't05_altura_medida1']

for var in missing_vars:
    if var in df.columns:
        print(f"\n{var}:")
        non_missing = df[var].dropna()
        if len(non_missing) > 0:
            print(f"  Valores válidos: {len(non_missing)}")
            print(f"  Range: {non_missing.min()} a {non_missing.max()}")
            print(f"  Quartis: Q1={non_missing.quantile(0.25):.2f}, Q2={non_missing.median():.2f}, Q3={non_missing.quantile(0.75):.2f}")
            
            # Valores extremos
            q1, q3 = non_missing.quantile([0.25, 0.75])
            iqr = q3 - q1
            outlier_threshold_low = q1 - 1.5 * iqr
            outlier_threshold_high = q3 + 1.5 * iqr
            
            outliers = non_missing[(non_missing < outlier_threshold_low) | (non_missing > outlier_threshold_high)]
            if len(outliers) > 0:
                print(f"  ⚠️  OUTLIERS (IQR): {len(outliers)} valores")
                print(f"      Limites: < {outlier_threshold_low:.2f} ou > {outlier_threshold_high:.2f}")
                print(f"      Valores extremos: {sorted(outliers.unique())}")

ANÁLISE DE RANGES E OUTLIERS - TODAS AS VARIÁVEIS NUMÉRICAS

regiao_Centro-Oeste:
  Count: 1960 (de 1960 - 0 missing)
  Min: 0
  Max: 1
  Mean: 0.24
  Median: 0.00
  Std: 0.42
  Valores únicos: [0, 1]
  Top 5 valores: {0: 1497, 1: 463}
--------------------------------------------------

regiao_Nordeste:
  Count: 1960 (de 1960 - 0 missing)
  Min: 0
  Max: 1
  Mean: 0.16
  Median: 0.00
  Std: 0.37
  Valores únicos: [0, 1]
  Top 5 valores: {0: 1646, 1: 314}
--------------------------------------------------

regiao_Norte:
  Count: 1960 (de 1960 - 0 missing)
  Min: 0
  Max: 1
  Mean: 0.17
  Median: 0.00
  Std: 0.37
  Valores únicos: [0, 1]
  Top 5 valores: {0: 1630, 1: 330}
--------------------------------------------------

regiao_Sudeste:
  Count: 1960 (de 1960 - 0 missing)
  Min: 0
  Max: 1
  Mean: 0.20
  Median: 0.00
  Std: 0.40
  Valores únicos: [0, 1]
  Top 5 valores: {0: 1568, 1: 392}
--------------------------------------------------

regiao_Sul:
  Count: 1960 (de 1960 - 0 missing)

In [5]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/8 - limpeza das variáveis/cs.csv')

# Analisar padrão de missing nas variáveis pré-natal
prenatal_vars = ['k04_prenatal_semanas', 'k05_prenatal_consultas']

print("ANÁLISE DE PADRÃO - MISSING PRÉ-NATAL")
print("="*50)

# 1. Casos com ambas variáveis missing
both_missing = df[prenatal_vars].isnull().all(axis=1)
print(f"Casos com AMBAS variáveis pré-natal missing: {both_missing.sum()}")

# 2. Casos com apenas uma missing
semanas_missing = df['k04_prenatal_semanas'].isnull()
consultas_missing = df['k05_prenatal_consultas'].isnull()

print(f"Apenas k04_prenatal_semanas missing: {(semanas_missing & ~consultas_missing).sum()}")
print(f"Apenas k05_prenatal_consultas missing: {(consultas_missing & ~semanas_missing).sum()}")

# 3. Verificar se existe informação de "NÃO FEZ PRÉ-NATAL"
# Valores mínimos podem indicar "não fez" (valor 0 ou muito baixo)
print(f"\nDistribuição k04_prenatal_semanas (valores válidos):")
semanas_validas = df['k04_prenatal_semanas'].dropna()
print(f"Min: {semanas_validas.min()}, Max: {semanas_validas.max()}")
print(f"Valores 0-2 semanas: {(semanas_validas <= 2).sum()} casos")

print(f"\nDistribuição k05_prenatal_consultas (valores válidos):")
consultas_validas = df['k05_prenatal_consultas'].dropna()
print(f"Min: {consultas_validas.min()}, Max: {consultas_validas.max()}")
print(f"Valores 0-1 consultas: {(consultas_validas <= 1).sum()} casos")

# 4. Correlação entre missing pré-natal e outras variáveis
print(f"\nCORRELAÇÃO MISSING PRÉ-NATAL COM OUTRAS VARIÁVEIS:")

# Missing pré-natal vs educação materna
educacao_baixa = df['educacao_Sem estudo'] + df['educacao_Fundamental']
correlacao_educ = both_missing.corr(educacao_baixa)
print(f"Missing pré-natal vs Educação baixa: {correlacao_educ:.3f}")

# Missing pré-natal vs renda
correlacao_renda = both_missing.corr(-df['q06_renda'])  # Negativo = menor renda
print(f"Missing pré-natal vs Menor renda: {correlacao_renda:.3f}")

# Missing pré-natal vs consulta pública
correlacao_publico = both_missing.corr(df['consulta_Público'])
print(f"Missing pré-natal vs Consulta pública: {correlacao_publico:.3f}")

ANÁLISE DE PADRÃO - MISSING PRÉ-NATAL
Casos com AMBAS variáveis pré-natal missing: 89
Apenas k04_prenatal_semanas missing: 30
Apenas k05_prenatal_consultas missing: 41

Distribuição k04_prenatal_semanas (valores válidos):
Min: 1.0, Max: 40.0
Valores 0-2 semanas: 122 casos

Distribuição k05_prenatal_consultas (valores válidos):
Min: 1.0, Max: 40.0
Valores 0-1 consultas: 5 casos

CORRELAÇÃO MISSING PRÉ-NATAL COM OUTRAS VARIÁVEIS:
Missing pré-natal vs Educação baixa: 0.059
Missing pré-natal vs Menor renda: 0.037
Missing pré-natal vs Consulta pública: 0.025


In [6]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/8 - limpeza das variáveis/cs.csv')

print("ANÁLISE PARA CÁLCULO DE IMC MATERNO")
print("="*50)

# Verificar se temos peso e altura maternos
peso_vars = [col for col in df.columns if 'peso' in col.lower()]
altura_vars = [col for col in df.columns if 'altura' in col.lower()]

print("Variáveis de peso disponíveis:")
for var in peso_vars:
    missing = df[var].isnull().sum()
    print(f"  {var}: {missing} missing ({missing/len(df)*100:.1f}%)")

print("\nVariáveis de altura disponíveis:")
for var in altura_vars:
    missing = df[var].isnull().sum()
    print(f"  {var}: {missing} missing ({missing/len(df)*100:.1f}%)")

# Verificar especificamente variáveis que podem ser peso/altura materno
print(f"\nvd_imc_mae missing: {df['vd_imc_mae'].isnull().sum()}")
print(f"t05_altura_medida1 missing: {df['t05_altura_medida1'].isnull().sum()}")

# Analisar se t05_altura_medida1 é altura materna
print(f"\nt05_altura_medida1 range: {df['t05_altura_medida1'].min():.1f} - {df['t05_altura_medida1'].max():.1f} cm")
print("(Range compatível com altura de adulto)")

# Verificar se temos peso materno atual ou peso final gestação
peso_materno_candidates = ['k07_peso_final', 'k06_peso_engravidar']
print(f"\nPossíveis pesos maternos:")
for var in peso_materno_candidates:
    if var in df.columns:
        missing = df[var].isnull().sum()
        range_val = f"{df[var].min():.1f} - {df[var].max():.1f} kg"
        print(f"  {var}: {missing} missing, range: {range_val}")

# Verificar overlap entre missing values
print(f"\nANÁLISE DE OVERLAP:")
imc_missing = df['vd_imc_mae'].isnull()
altura_missing = df['t05_altura_medida1'].isnull() 
peso_final_missing = df['k07_peso_final'].isnull()

print(f"Casos com IMC missing MAS com altura E peso disponíveis:")
casos_calcular = imc_missing & ~altura_missing & ~peso_final_missing
print(f"  Total: {casos_calcular.sum()} casos")

if casos_calcular.sum() > 0:
    print(f"  Isso representa {casos_calcular.sum()/imc_missing.sum()*100:.1f}% dos IMC missing")
    
    # Teste de cálculo
    print(f"\nTESTE DE CÁLCULO IMC:")
    subset = df[casos_calcular].head(3)
    for idx, row in subset.iterrows():
        peso = row['k07_peso_final']
        altura = row['t05_altura_medida1'] / 100  # converter cm para metros
        imc_calculado = peso / (altura ** 2)
        print(f"  Caso {idx}: {peso}kg / ({altura:.2f}m)² = IMC {imc_calculado:.2f}")

ANÁLISE PARA CÁLCULO DE IMC MATERNO
Variáveis de peso disponíveis:
  h02_peso: 0 missing (0.0%)
  k06_peso_engravidar: 140 missing (7.1%)
  k07_peso_final: 137 missing (7.0%)

Variáveis de altura disponíveis:
  h03_altura: 0 missing (0.0%)
  t05_altura_medida1: 22 missing (1.1%)

vd_imc_mae missing: 279
t05_altura_medida1 missing: 22

t05_altura_medida1 range: 137.2 - 183.0 cm
(Range compatível com altura de adulto)

Possíveis pesos maternos:
  k07_peso_final: 137 missing, range: 38.0 - 138.0 kg
  k06_peso_engravidar: 140 missing, range: 35.0 - 133.0 kg

ANÁLISE DE OVERLAP:
Casos com IMC missing MAS com altura E peso disponíveis:
  Total: 245 casos
  Isso representa 87.8% dos IMC missing

TESTE DE CÁLCULO IMC:
  Caso 0: 56.0kg / (1.56m)² = IMC 22.86
  Caso 1: 70.0kg / (1.62m)² = IMC 26.74
  Caso 4: 65.0kg / (1.67m)² = IMC 23.22


In [7]:
import pandas as pd
import numpy as np

# Carregar dados
df = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/8 - limpeza das variáveis/cs.csv')

print("ANÁLISE DE CASOS COMPLETOS (SEM IMPUTAÇÃO)")
print("="*60)

# Separar X e y
X = df.drop(['aleitamento_materno_exclusivo', 'id_anon'], axis=1)
y = df['aleitamento_materno_exclusivo']

print(f"Dataset original: {len(df)} observações, {X.shape[1]} features")

# Identificar casos com ANY missing
casos_com_missing = X.isnull().any(axis=1)
casos_completos = ~casos_com_missing

print(f"\nCasos com algum missing: {casos_com_missing.sum()}")
print(f"Casos completos: {casos_completos.sum()}")
print(f"Taxa de casos completos: {casos_completos.mean()*100:.1f}%")
print(f"Perda de amostra: {casos_com_missing.sum()} casos ({casos_com_missing.mean()*100:.1f}%)")

# Análise por variável de missing
print(f"\nContribuição de cada variável para a perda:")
missing_vars = X.isnull().sum()
missing_vars = missing_vars[missing_vars > 0].sort_values(ascending=False)

for var, count in missing_vars.items():
    # Quantos casos adicionais seriam salvos se esta variável não tivesse missing
    casos_perdidos_por_esta_var = X[var].isnull().sum()
    print(f"  {var}: {count} missing ({count/len(df)*100:.1f}%)")

# Dataset final se usarmos apenas casos completos
X_completo = X[casos_completos]
y_completo = y[casos_completos]

print(f"\n" + "="*60)
print("DATASET FINAL SEM IMPUTAÇÃO:")
print(f"Observações: {len(X_completo)} (de {len(df)} originais)")
print(f"Features: {X_completo.shape[1]}")
print(f"Target balanceamento:")
balanceamento = y_completo.value_counts()
for valor, count in balanceamento.items():
    print(f"  Classe {valor}: {count} ({count/len(y_completo)*100:.1f}%)")

print(f"\nPERDA LÍQUIDA: {len(df) - len(X_completo)} observações ({(len(df) - len(X_completo))/len(df)*100:.1f}%)")

# Verificar se a perda é aleatória ou enviesada
print(f"\nANÁLISE DE VIÉS NA PERDA:")
print("Comparando características entre casos completos vs. casos com missing:")

# Algumas variáveis chave para verificar viés
vars_check = ['bb04_idade_da_mae', 'educacao_Superior', 'q06_renda', 'regiao_Sudeste']

for var in vars_check:
    if var in df.columns:
        media_completos = df.loc[casos_completos, var].mean()
        media_missing = df.loc[casos_com_missing, var].mean()
        diferenca = abs(media_completos - media_missing)
        print(f"  {var}: Completos={media_completos:.2f}, Missing={media_missing:.2f}, Diff={diferenca:.2f}")

print(f"\n✓ Dataset pronto para análise: {len(X_completo)} casos, {X_completo.shape[1]} features")

ANÁLISE DE CASOS COMPLETOS (SEM IMPUTAÇÃO)
Dataset original: 1960 observações, 118 features

Casos com algum missing: 632
Casos completos: 1328
Taxa de casos completos: 67.8%
Perda de amostra: 632 casos (32.2%)

Contribuição de cada variável para a perda:
  vd_imc_mae: 279 missing (14.2%)
  vd_anthro_zwfl: 172 missing (8.8%)
  vd_zimc: 171 missing (8.7%)
  k08_quilos: 164 missing (8.4%)
  k06_peso_engravidar: 140 missing (7.1%)
  k07_peso_final: 137 missing (7.0%)
  k05_prenatal_consultas: 130 missing (6.6%)
  k04_prenatal_semanas: 119 missing (6.1%)
  tempo_primeira_mamada_horas: 83 missing (4.2%)
  k01_gestacoes: 40 missing (2.0%)
  k02_filhos_vivos: 40 missing (2.0%)
  t05_altura_medida1: 22 missing (1.1%)

DATASET FINAL SEM IMPUTAÇÃO:
Observações: 1328 (de 1960 originais)
Features: 118
Target balanceamento:
  Classe 1: 680 (51.2%)
  Classe 0: 648 (48.8%)

PERDA LÍQUIDA: 632 observações (32.2%)

ANÁLISE DE VIÉS NA PERDA:
Comparando características entre casos completos vs. casos com

# Justificativa Metodológica: Rejeição da Exclusão Listwise

## Contexto da Decisão

Na análise do dataset ENANI-2019 para predição de aleitamento materno exclusivo, consideramos inicialmente a exclusão listwise (remoção de casos com qualquer valor ausente) como estratégia para lidar com missing values. Após análise sistemática, rejeitamos esta abordagem pelas seguintes razões metodológicas e estatísticas.

## Análise Quantitativa da Perda Amostral

### Impacto na Amostra
- **Dataset original**: 1.960 observações
- **Casos completos**: 1.328 observações
- **Perda amostral**: 632 casos (32,2%)
- **Razão observações/features**: 11,3:1 (1.328 obs ÷ 118 features)

### Distribuição dos Missing Values
A perda não é uniforme entre as variáveis:
- `vd_imc_mae`: 279 missing (14,2%)
- `vd_anthro_zwfl`: 172 missing (8,8%) 
- `vd_zimc`: 171 missing (8,7%)
- `k08_quilos`: 164 missing (8,4%)
- Outras 8 variáveis: 1,1% a 7,1% missing

## Viés Sistemático Identificado

### Análise de Não-Aleatoriedade
A exclusão listwise introduz **viés sistemático** significativo, violando a premissa MCAR (Missing Completely at Random):

| Característica | Casos Completos | Casos com Missing | Diferença |
|----------------|-----------------|-------------------|-----------|
| Idade materna (anos) | 28,4 | 24,7 | **+3,7 anos** |
| Educação superior (%) | 21,0% | 10,0% | **+11 pp** |
| Renda familiar (R$) | 2.433 | 1.871 | **+R$ 562** |
| Região Sudeste (%) | 20,0% | 20,0% | 0,0 |

### Interpretação do Padrão de Viés
Os dados revelam que casos com missing values concentram-se sistematicamente em:
- **Mães mais jovens**: 3,7 anos de diferença média
- **Menor escolaridade**: Diferença de 11 pontos percentuais em educação superior
- **Menor renda familiar**: R$ 562 de diferença média
- **Perfil socioeconômico vulnerável**: Concentração em estratos sociais inferiores

## Implicações Metodológicas

### 1. Redução do Poder Estatístico
- Perda de 32% da amostra reduz significativamente o poder estatístico
- Razão N:p de 11,3:1, embora aceitável, está no limite inferior recomendado
- Menor capacidade de detectar padrões em subgrupos populacionais

### 2. Comprometimento da Validade Externa
- **Viés de seleção**: Amostra final não representativa da população brasileira
- **Generalização limitada**: Resultados aplicáveis principalmente a mães de maior nível socioeconômico
- **Subrepresentação**: Grupos vulneráveis (mães jovens, baixa escolaridade, baixa renda) inadequadamente representados

### 3. Violação de Pressupostos Estatísticos
- **MCAR violado**: Missing values não são completamente aleatórios
- **Viés de estimação**: Parâmetros do modelo enviesados em direção a características de maior nível socioeconômico
- **Intervalo de confiança**: Subestimação da variabilidade populacional real

## Considerações para Revisão por Pares

### Vulnerabilidades Metodológicas
A exclusão listwise seria alvo de críticas específicas dos revisores:

1. **Questão de Equidade**: Exclusão sistemática de populações vulneráveis contraria princípios de saúde pública
2. **Rigor Metodológico**: Perda de 1/3 da amostra sem justificativa técnica adequada
3. **Validade dos Achados**: Conclusões limitadas a subpopulação de melhor condição socioeconômica
4. **Replicabilidade**: Dificuldade de reprodução em contexts de maior vulnerabilidade social

### Padrões da Literatura Científica
Estudos epidemiológicos contemporâneos exigem:
- Preservação máxima da representatividade amostral
- Tratamento explícito de missing values
- Análises de sensibilidade para robustez metodológica
- Consideração de equidade populacional

## Conclusão Metodológica

A exclusão listwise foi rejeitada devido a:

1. **Perda amostral significativa** (32,2%)
2. **Viés sistemático robusto** contra populações vulneráveis  
3. **Comprometimento da validade externa**
4. **Incompatibilidade com padrões contemporâneos** de pesquisa epidemiológica

## Abordagem Alternativa Adotada

Com base nesta análise, optamos por estratégia de **imputação estatística**, que:
- Preserva representatividade populacional
- Mantém poder estatístico adequado
- Permite análise de sensibilidade
- Alinha-se aos padrões metodológicos atuais
- Oferece maior robustez para revisão por pares

A implementação da imputação será documentada com igual rigor metodológico, incluindo análises comparativas e testes de robustez para garantir a integridade científica dos resultados.

In [9]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

# Carregar dados originais
df = pd.read_csv('/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/8 - limpeza das variáveis/cs.csv')

print("IMPUTAÇÃO COM MEDIANA - ESTRATÉGIA CONSERVADORA")
print("="*50)

# Separar variáveis
X = df.drop(['aleitamento_materno_exclusivo', 'id_anon'], axis=1)
y = df['aleitamento_materno_exclusivo']

# Identificar variáveis com missing
missing_vars = X.columns[X.isnull().any()].tolist()
print(f"Variáveis com missing values: {len(missing_vars)}")
for var in missing_vars:
    missing_count = X[var].isnull().sum()
    missing_pct = missing_count / len(X) * 100
    print(f"  {var}: {missing_count} ({missing_pct:.1f}%)")

# Aplicar imputação por mediana
print(f"\nAplicando imputação por mediana...")
imputer = SimpleImputer(strategy='median')

# Imputar apenas colunas numéricas com missing
X_imputed = X.copy()
for var in missing_vars:
    if X[var].dtype in ['float64', 'int64']:  # Apenas numéricas
        X_imputed[[var]] = imputer.fit_transform(X[[var]])
        print(f"  ✓ {var}: mediana = {X[var].median():.2f}")

# Verificar se ainda há missing values
remaining_missing = X_imputed.isnull().sum().sum()
print(f"\nMissing values após imputação: {remaining_missing}")

if remaining_missing > 0:
    print("Variáveis ainda com missing:")
    still_missing = X_imputed.isnull().sum()
    for var, count in still_missing[still_missing > 0].items():
        print(f"  {var}: {count}")

# Recombinar dataset completo
df_final = pd.concat([
    df[['id_anon', 'aleitamento_materno_exclusivo']], 
    X_imputed
], axis=1)

# Salvar resultado
output_path = '/Users/marcelosilva/Desktop/copia2 - artigo peer/predição_amamentação/9 - Aplicando Lasso, ridge e Elastic/Lasso.csv'
df_final.to_csv(output_path, index=False)

print(f"\n✓ Dados imputados salvos em: Lasso.csv")
print(f"✓ Shape final: {df_final.shape}")
print(f"✓ Dataset pronto para regularização (Lasso, Ridge, Elastic Net)")

# Verificação final
print(f"\nVERIFICAÇÃO FINAL:")
print(f"Observações: {len(df_final)}")
print(f"Features: {df_final.shape[1] - 2}")  # -2 para id e target
print(f"Target balanceamento: {y.value_counts().to_dict()}")
print(f"Missing values total: {df_final.isnull().sum().sum()}")

IMPUTAÇÃO COM MEDIANA - ESTRATÉGIA CONSERVADORA
Variáveis com missing values: 12
  k01_gestacoes: 40 (2.0%)
  k02_filhos_vivos: 40 (2.0%)
  k04_prenatal_semanas: 119 (6.1%)
  k05_prenatal_consultas: 130 (6.6%)
  k06_peso_engravidar: 140 (7.1%)
  k07_peso_final: 137 (7.0%)
  k08_quilos: 164 (8.4%)
  tempo_primeira_mamada_horas: 83 (4.2%)
  t05_altura_medida1: 22 (1.1%)
  vd_zimc: 171 (8.7%)
  vd_anthro_zwfl: 172 (8.8%)
  vd_imc_mae: 279 (14.2%)

Aplicando imputação por mediana...
  ✓ k01_gestacoes: mediana = 2.00
  ✓ k02_filhos_vivos: mediana = 2.00
  ✓ k04_prenatal_semanas: mediana = 8.00
  ✓ k05_prenatal_consultas: mediana = 9.00
  ✓ k06_peso_engravidar: mediana = 61.50
  ✓ k07_peso_final: mediana = 72.00
  ✓ k08_quilos: mediana = 10.15
  ✓ tempo_primeira_mamada_horas: mediana = 1.00
  ✓ t05_altura_medida1: mediana = 160.00
  ✓ vd_zimc: mediana = 0.10
  ✓ vd_anthro_zwfl: mediana = 0.20
  ✓ vd_imc_mae: mediana = 25.91

Missing values após imputação: 0

✓ Dados imputados salvos em: Lass